In [68]:
import os
import json
import cohere
import re
import logging
from datetime import datetime

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

co = cohere.Client(api_key=os.getenv('COHERE_API_KEY'))

In [69]:
log_file_name = ""


def ensure_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def log(message):
    global log_file_name
    ensure_directory("logs")
    with open(log_file_name, "a") as file:
        file.write(f"{datetime.now()}\n{message}\n")

In [70]:
conversation_price = 0
conversation_ct = 0
conversation_pt = 0
conversation_tt = 0


def get_conversation_price(new_completion):
    global conversation_price, conversation_ct, conversation_pt, conversation_tt
    USD_price_in_COP = 4100

    # gpt4o
    ct_price = 0.000015
    pt_price = 0.000005

    ct = new_completion.usage.completion_tokens
    pt = new_completion.usage.prompt_tokens
    tt = new_completion.usage.total_tokens
    usd_total_price = (ct * ct_price) + (pt * pt_price)
    cop_total_price = usd_total_price * USD_price_in_COP
    conversation_ct += ct
    conversation_pt += pt
    conversation_tt += tt
    conversation_usd_total_price = (conversation_ct * ct_price) + (
        conversation_pt * pt_price
    )
    conversation_cop_total_price = conversation_usd_total_price * USD_price_in_COP

    print(
        f"💰 Conversation price: ${conversation_cop_total_price} COP (In: {conversation_pt}, Out: {conversation_ct}, Total token: {conversation_tt})\n"
        f"   This message: ${cop_total_price} COP (In: {pt}, Out: {ct}, Total token: {tt}) "
        f""
    )
    log(
        f"💰 Conversation price: ${conversation_cop_total_price} COP (In: {conversation_pt}, Out: {conversation_ct}, Total token: {conversation_tt})\n "
        f"   This message: ${cop_total_price} COP (In: {pt}, Out: {ct}, Total token: {tt}) "
        f""
    )

    return

In [71]:
### 🔑 Definir system prompt y tools

messages = []
user_input = ""

query_prompt = "Utiliza esta herramienta si necesitas más fuentes para responder al usuario con precisión. Genera dos consultas de búsqueda basadas en lo que el usuario te ha preguntado: una consulta de texto completo utilizando sintaxis Lucene y una consulta semántica. Cada una debe contener la idea completa y fuentes específicas si son necesarias. Si la pregunta del usuario aborda cuándo, cómo o qué información, debes incluir eso en las consultas, ya que afectará los resultados. Evitar hacer búsquedas de coincidencias exactas de palabras, ya que es probables que los textos esten mal transcritos. Utiliza búsquedas por campo, especialmente siempre busca en el campo 'content' y utiliza los campos 'title' y 'author' si son útiles para obtener el fragmento correcto del documento. La consulta semántica debe ser más general y capturar el significado completo de lo que el usuario está preguntando. Si necesitas diferentes fuentes, busca cada una con un llamado a esta tool"
# query_prompt = "Use this tool if you need more sources to respond the user acurrately. Generate a search query in Spanish based on what the user asked you. Each query should contain the whole idea and specific sources if they are needed. If the user question adresses a when, how or what information, you should keep that in the query because that will affect what you will get. The query will be used in a hybrid search (semantic and full text) so keep it short but make sure to include all the main keywords and infromation needed to give the necesary context to the query. Try to always include at least one concept followed by one specific source. Don't include more than 1 source. You can include the various ways of refering the same source eg: 'homicidio artículo 103 ley 599 del 2000 código penal'. If you need multiple searches, call this tools as many times as needed."
# Debes generar un query semánticamente completo

# search_legal_info = {
#     "type": "function",
#     "function": {
#         "name": "search_legal_info",
#         "description": query_prompt,
#         "parameters": {
#             "type": "object",
#             "strict": "true",
#             "properties": {
#                 "search_query": {
#                     "type": "string",
#                     "description": "The text search query for supporting legal documents. eg: 'homicidio artículo 103 ley 599 del 2000', 'cuándo se consuma el hurto artículo 239 ley 599 del 2000'",
#                 },
#                 "search_type": {
#                     "enum": ["exact", "interpret"],
#                     "type": "string",
#                     "description": "If the query is for and exact citation or for a more concept interpretation",
#                 },
#                 "number_of_chunks_needed": {
#                     "type": "integer",
#                     "description": "A number between 3 and 10 depending on how many 500 tokens chunks of documents you want to retrieve out of this query",
#                 },
#             },
#             "required": ["search_query", "search_type", "number_of_chunks_needed"],
#         },
#     },
# }
search_legal_info = {
    "type": "function",
    "function": {
        "name": "search_legal_info",
        "description": query_prompt,
        "parameters": {
            "type": "object",
            "strict": "true",
            "properties": {
                "full_text_query": {
                    "type": "string",
                    "description": "La consulta de texto completo utilizando sintaxis Lucene. Debe enfocarse en los campos 'content', 'title' y 'author' para encontrar los fragmentos correctos, evitando depender de coincidencias exactas de palabras debido a posibles errores de transcripción en los documentos. Ejemplo: 'content:causales de agravación artículo 267 AND title:ley 599 del 2000'",
                },
                "semantic_query": {
                    "type": "string",
                    "description": "La consulta semántica que captura el significado completo de lo que el usuario está preguntando. Debe ser más general y no utilizar sintaxis específica de campos o operadores lógicos. Ejemplo: 'causales de agravación de la estafa artículo 267 ley 599 del 2000'",
                },
            },
            "required": ["semantic_query", "full_text_query"],
        },
    },
}
get_next_chunk = {
    "type": "function",
    "function": {
        "name": "get_next_chunk",
        "description": "Use this tool to get when an important sources comes incomplete or the content is cut off. This tool will help you retrieve the following section of that given source.",
        "parameters": {
            "type": "object",
            "properties": {
                "source_id": {
                    "type": "string",
                    "description": "the unique identifier of the chunk that is incomplete. eg:'20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30'",
                },
            },
            "required": ["source_id"],
        },
    },
}

# response_system_template = f'Eres Ariel, un asistente para la investigación legal en jurisprudencia colombiana. Tienes acceso a una base de datos con cientos de miles de documentos legales, la cuál consultarás para responder a la solicitud del usuario a través de la tool "search_legal_info". Tu query debe ser muy específico pero incluir toda la información necesaria para obtener las fuentes pertinentes para responder tu pregunta. Si es necesario que generes distintos queries para obtener distintos tipos de documentos, lo puedes hacer. Debes Ser lo más detallado y preciso posible en tus respuestas, siempre utilizando exclusivamente la información propocionada en las fuentes consultadas. Cita el máximo número de fuentes que consideres aplicable a la solicitud del usuario. Nunca des información que no haya sido extraida directamente de las fuentes. En caso que las fuentes no te ayuden a responder la pregunta, responde "No encuentro información con esos términos, ¿puedes reformular tu consulta?". En la base de datos encontrarás fragmentos de documentos legales de máximo 500 tokens, así que si identificados que uno de los fragmentos es necesario para responder la pregunta, pero el fragmento está cortado por la parte que necesitabas, ejecuta "get_next_chunk" para obtener el siguiente fragmento de esa fuente. Interpreta las fuentes dependiendo de su categoría: 1. Constitucional: Es la norma máxima y más importante establecida por el gobierno del país. 2. Legal: Es la que le sigue en importancia y son los decretos, códigos y leyes oficiales del país establecida por los ministerios. 3. Infralegal: Le sigue en importancia a la categoría Legal. 4. Jurisprudencia: Son los documentos de juicios reales y de cómo se ha ejercido la ley en el pasado. 5. Doctrina: Son documentos o libros escritos por autores del derecho que no son parte de la ley pero ayudan a darle una interpetación a la ley. Dale mayor importancia a los fragmentos más actuales. Escribe tus respuesta en formato HTML, pero sin incluir los tags "```html" al principio o final principio de tu respuesta. Utiliza los corchetes para referenciar la fuente con el ID del fragmento referenciado sin manipularlo e ninguna forma, por ejemplo: [12345_id_de_ejemplo]. No combines fuentes, lista cada fuente por separado. Una vez recibas las fuentes, si consideras que necesitas hacer otra búsqueda, hazla. Si uno de los extractos que vas a usar cita un artículo, ley u otro tipo de fuente, asegúrate de buscar primero esa fuente directamente para que tengas la información de primera mano. Siempre intenta citar la fuente primaria de la información. Nunca confundas leyes o artículos, si el usuario preguntó por una ley, ignora los extractos que hablen de otras leyes.'

response_system_template = f'Eres Ariel, un asistente para la investigación legal en jurisprudencia colombiana. Sigue estas instrucciones al responder:\n\nConsulta de Fuentes:\n\nUtiliza la herramienta "search_legal_info" para buscar información relevante.\nCrea consultas específicas que incluyan toda la información necesaria.\nSi es necesario, genera múltiples consultas para obtener diferentes tipos de documentos.\nRedacción de Respuestas:\n\nSé detallado y preciso, utilizando exclusivamente la información de las fuentes proporcionadas.\nNo incluyas información que no haya sido extraída directamente de las fuentes.\nCitación de Fuentes:\n\nCita el mayor número de fuentes aplicables.\nUtiliza corchetes para referenciar la fuente con el ID del fragmento sin modificarlo, por ejemplo: [12345_id_de_ejemplo].\nNo combines fuentes; lista cada fuente por separado.\nFormato de Respuesta:\n\nEscribe tus respuestas en formato HTML, sin incluir los tags "```html" al principio o al final.\nUtiliza etiquetas HTML estándar como <p>, <strong>, <em>, etc.\nManejo de Fragmentos Incompletos:\n\nSi un fragmento es necesario pero está incompleto, utiliza "get_next_chunk" con el source_id correspondiente para obtener el siguiente fragmento.\nInterpretación de Categorías:\n\nPrioriza las fuentes según su categoría:\nConstitucional\nLegal\nInfralegal\nJurisprudencia\nDoctrina\nDa mayor importancia a los fragmentos más actuales.\nPrecisión Legal:\n\nSi un extracto cita otra fuente (artículo, ley, etc.), búscala directamente para obtener información de primera mano.\nNo confundas leyes o artículos; si el usuario pregunta por una ley específica, ignora extractos de otras leyes.\nConsultas sin Resultados:\n\nSi las fuentes no ayudan a responder la pregunta, responde: "No encuentro información con esos términos, ¿puedes reformular tu consulta?"'

system_message = {"role": "system", "content": response_system_template}

In [72]:
def format_search_results(docs_list):
    documents = []
    for document in docs_list:
        captions: QueryCaptionResult = document["@search.captions"]
        captions_text = " // ".join([caption.text for caption in captions]) if captions is not None else ""
        doc_formatted = {
            "position" : len(documents) + 1,
            "score": document["@search.score"],
            "rerank": document["@search.reranker_score"],
            "captions": captions_text,
            "id": document["id"],
            "title": document["title"],
            "author": document["author"],
            "keywords": document["keywords"],
            "category": document["category"],
            "page": document["page"],
            "year": document["year"],
            "has_copyright": document["has_copyright"],
            "file_path": document["file_path"],
            "external_id": document["external_id"],
            "content": document["content"],
        }
        documents.append(doc_formatted)
    log(
        f">>>>>>>> format_search_results:\n"
        f"{json.dumps(documents, indent=4)}"
        )
    return documents

def format_reranked_results(docs_list):
    documents = []
    for document in docs_list:
        doc_formatted = {
            "position" : len(documents) + 1,
            "id": document.document.id,
            "relevance": document.relevance_score,
            "score": document.document.score,
            "rerank": document.document.rerank,
            "captions": document.document.captions,
            "title": document.document.title,
            "author": document.document.author,
            "keywords": document.document.keywords,
            "category": document.document.category,
            "page": document.document.page,
            "year": document.document.year,
            "has_copyright": document.document.has_copyright,
            "file_path": document.document.file_path,
            "external_id": document.document.external_id,
            "content": document.document.content,
        }
        documents.append(doc_formatted)
    log(
        f">>>>>>> format_reranked_results:\n"
        f"{json.dumps(documents, indent=4)}"        
    )
    return documents

def filter_docs(doc_list):
    filtered_docs = []
    contents = [doc["content"] for doc in doc_list]
    
    for doc in doc_list:
        score = doc["score"] if isinstance(doc["score"], float) else 0
        rerank = doc["rerank"] if isinstance(doc["rerank"], float) else 0
        relevance = doc.get("relevance", 1)
        if (
            contents.count(doc["content"]) == 1
            and (relevance > 0.65
            and rerank > 2
            or score > 0.025)
        ):
            filtered_docs.append(doc)
    print(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...")
    log(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...\n"
    f"{json.dumps(filtered_docs, indent=4)}")
    return filtered_docs

def add_doc_to_context(docs_list):
    sources = ""
    counter = 0
    
    for document in docs_list:
        counter += 1
        sources += (
            f"Fuente #{counter}\n"
            f"ID: {document["id"]}\n"
            f"Título: {document["title"]}\n"
            f"Autor: {document["author"]}\n"
            f"Año de publicación: {document["year"]}\n"
            f"Área Legal: {document["keywords"]}\n"
            f"Tipo de Documento: {document["category"]}\n"
            f"Página: {document["page"]}\n"
            f"Extracto: {document["content"]}\n"
            f"\n\n"
        )      
    log(
        f"To add in Context: \n"
        f"{json.dumps({"sources":sources}, indent=4)}"
        )
    return sources
    

In [73]:
def search_for_chunks(text_query, vector_query, rerank_query):
    log(
        f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
        f">> 🔎 Buscando documentos Semantic Query: {rerank_query} ...\n"
    )
    print(
        f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
        f">> 🔎 Buscando documentos Semantic Query: {rerank_query} ...\n"
    )
    results_num = 50
    results = search_client.search(
        search_text=text_query,
        vector_queries=[
            {
                "vector": vector_query,
                "k": results_num,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        top=results_num,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal2",
        scoring_parameters=[
            "categoryBoost-Constitucional,Legal,Infralegal,Jurisprudencia,Doctrina"
        ],
    )

    # Convertir en Lucene full text syntax y pedir otro query semántico
    #

    results_formatted = format_search_results(results)
    restults_filtered = filter_docs(results_formatted)

    log(f">> ⭐️ Aplicando reorganización semántica: {rerank_query} ...")
    print(f">> ⭐️ Aplicando reorganización semántica ...")
    reranked_docs = co.rerank(
        model="rerank-multilingual-v3.0",
        top_n=10,
        query=rerank_query,
        documents=restults_filtered,
        return_documents=True,
        rank_fields=["content", "title", "author", "keywords", "category"],
    )
    reranked_docs_formatted = format_reranked_results(reranked_docs.results)
    reranked_docs_filtered = filter_docs(reranked_docs_formatted)

    # reranked_docs_filtered = restults_filtered[:10]

    return reranked_docs_filtered


def get_next_chunks(id):
    log(f">> ⏩️ Buscando los chunks siguientes de {id} ...")
    print(f">> ⏩️ Buscando los chunks siguientes de {id} ...")
    pattern = r"^(.*_chunk)(\d+)$"
    match = re.search(pattern, id)

    if match:
        prefix = match.group(1)
        chunk_number = int(match.group(2))
        next_chunks_result = search_client.search(
            filter=f"id eq '{prefix}{chunk_number + 1}' or id eq '{prefix}{chunk_number + 2}'",
            top=2,
        )
        return format_search_results(list(next_chunks_result))


def get_next_chunk_tool(source_id):
    next_chunks = get_next_chunks(source_id)
    log(f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}")
    return f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}"


def search_legal_info_tool(text_query, semantic_query):
    category_weight = " AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"

    vector_query = embeddings_client.generate_embeddings(content=semantic_query)

    results = search_for_chunks(
        text_query=text_query + category_weight,
        rerank_query=semantic_query,
        vector_query=vector_query,
    )
    return f"Extractos de documentos encontrados: \n{add_doc_to_context(results)}"

In [74]:
def generate_completion():
    global messages
    log(f"{json.dumps(messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        messages=messages,
        tools=[get_next_chunk, search_legal_info],
        temperature=0.2,
        n=1,
        tool_choice="auto",
    )
    log(response)
    get_conversation_price(response)
    return response

In [75]:
def append_message(new_message):
    global messages
    conversation = messages

    if isinstance(new_message, dict):
        role = new_message.get("role")
        content = new_message.get("content")
    else:
        role = new_message.role
        content = new_message.content

    if role == "assistant":
        if new_message.tool_calls:
            tool_calls = new_message.tool_calls
            tool_calls_formatted = []
            for tool_call in tool_calls:
                tool_calls_formatted.append(
                    {
                        "id": tool_call.id,
                        "function": {
                            "arguments": str(json.loads(tool_call.function.arguments)),
                            "name": tool_call.function.name,
                        },
                        "type": "function",
                    }
                )
            conversation.append(
                {
                    "role": "assistant",
                    "tool_calls": tool_calls_formatted,
                },
            )
            return conversation  ## Assistant call tools
        elif content:
            conversation.append({"role": "assistant", "content": content})
            return conversation  ## Assistant talk
    elif role == "tool":
        conversation.append(new_message)
        return conversation  ## Tool reponse
    else:
        return conversation

In [76]:
def call_tools(tool_calls):
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        if tool_name == "get_next_chunk":
            content = get_next_chunk_tool(tool_args.get("source_id"))
        elif tool_name == "search_legal_info":
            content = search_legal_info_tool(
                text_query=tool_args.get("full_text_query"),
                semantic_query=tool_args.get("semantic_query"),
            )

        new_tool_message = {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": tool_name,
            "content": content,
        }

        append_message(new_message=new_tool_message)

    print(">> 🤖 Generando respuesta ...")
    new_assistant_completion = generate_completion()
    new_assistant_message = new_assistant_completion.choices[0].message
    append_message(new_message=new_assistant_message)

    if new_assistant_message.content:
        return print("💬 Assistant:" + new_assistant_message.content)
    else:
        return call_tools(
            tool_calls=new_assistant_message.tool_calls,
        )

In [77]:
def run_conversation(user_prompt=None):
    if user_prompt:
        get_answer(user_prompt)
    else:
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Exiting chat...")
                break
            get_answer(user_input)


def get_answer(user_prompt):
    global messages, log_file_name, user_input

    user_input = user_prompt

    current_time = datetime.now().strftime("%m-%d %H:%M")
    log_file_name = f"logs/{current_time} - {user_input[:50]}.log"

    print("🙎‍♂️ User: " + user_input)
    log(f"User input: {user_input}")
    new_user_message = {"role": "user", "content": user_input}

    messages = [
        system_message,
        new_user_message,
    ]

    print(">> 🤖 Generando respuesta ...")
    response = generate_completion()
    response_message = response.choices[0].message

    messages = append_message(new_message=response_message)

    tool_calls = response_message.tool_calls
    if tool_calls:
        call_tools(tool_calls=tool_calls)
    elif response_message.content:
        print("💬 Assistant:", response_message.content)
    else:
        print("An error ocurred during completion generation - response: ", response)

In [78]:
message = []
run_conversation("¿Cuáles son las causales genéricas de atenuación punitiva?")

🙎‍♂️ User: ¿Cuáles son las causales genéricas de atenuación punitiva?
>> 🤖 Generando respuesta ...
💰 Conversation price: $26.117000000000004 COP (In: 866, Out: 136, Total token: 1002)
   This message: $26.117000000000004 COP (In: 866, Out: 136, Total token: 1002) 
>> 🔎 Buscando documentos: content:causales genéricas de atenuación punitiva AND title:ley 599 del 2000 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🧹 Docs filtrados. De 50 pasaron 16 ...
>> 🔎 Buscando documentos: content:causales genéricas de atenuación punitiva AND title:jurisprudencia AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🧹 Docs filtrados. De 50 pasaron 18 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $347.14700000000005 COP (In: 15302, Out: 544, Total token: 15846)
   This message: $321.03000000000003 COP (In: 14436, Out: 408, Total token: 14844) 
💬 Assistant:<p>Las causales genéricas de atenuación punitiva en el sistema penal

In [ ]:
messages = []
run_conversation("¿Qué dice el artículo 103 del código penal?")

🙎‍♂️ User: ¿Qué dice el artículo 103 del código penal?
>> 🤖 Generando respuesta ...
💰 Conversation price: $268.93950000000007 COP (In: 10803, Out: 772, Total token: 11575)
   This message: $21.443 COP (In: 923, Out: 41, Total token: 964) 
>> 🔎 Buscando documentos: artículo 103 ley 599 del 2000 código penal ...
>> 🧹 Docs filtrados. De 50 pasaron 30 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 7 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $363.54699999999997 COP (In: 15334, Out: 800, Total token: 16134)
   This message: $94.6075 COP (In: 4531, Out: 28, Total token: 4559) 
>> ⏩️ Buscando los chunks siguientes de 20240814181609ley599de2000pdf_chunk69 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $483.3695 COP (In: 20918, Out: 887, Total token: 21805)
   This message: $119.82250000000002 COP (In: 5584, Out: 87, Total token: 5671) 
💬 Assistant:El artículo 103 del Código Penal colombiano, contenido en la Ley 599 de 2000, establece lo 

In [ ]:
# messages = []
# run_conversation(
#     "Daniel Andrés Fúquenes Barriga, en su condición de auxiliar de la justicia y secuestre, recaudó una suma de dinero por concepto de arrendamiento, específicamente $684,000 entre febrero y julio de 2015. Este dinero debía ser entregado a su dueño o poseedor, pero Fúquenes Barriga retuvo la suma para sí mismo, incumpliendo con la obligación de devolverla. Este acto de retención y apropiación del dinero, que se le había confiado por un título no traslativo de dominio, ¿Qué delito cometió?"
# )

In [ ]:
messages = []
run_conversation("Dame el artículo 55 del código penal")

🙎‍♂️ User: Dame el artículo 55 del código penal
>> 🤖 Generando respuesta ...
💰 Conversation price: $187.88250000000002 COP (In: 7854, Out: 437, Total token: 8291)
   This message: $20.992 COP (In: 922, Out: 34, Total token: 956) 
>> 🔎 Buscando documentos: artículo 55 código penal ...
>> 🧹 Docs filtrados. De 50 pasaron 14 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 0 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $210.08400000000003 COP (In: 8832, Out: 472, Total token: 9304)
   This message: $22.201500000000003 COP (In: 978, Out: 35, Total token: 1013) 
>> ⏩️ Buscando los chunks siguientes de 20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $247.49650000000003 COP (In: 9880, Out: 731, Total token: 10611)
   This message: $37.4125 COP (In: 1048, Out: 259, Total token: 1307) 
💬 Assistant:El artículo 55 del Código Penal colombiano establece lo siguiente:

**Artículo 55. Circ

In [ ]:
messages = []
run_conversation("¿Qué se entiende por “documento” en el derecho penal?")

🙎‍♂️ User: ¿Qué se entiende por “documento” en el derecho penal?
>> 🤖 Generando respuesta ...
💰 Conversation price: $504.62800000000004 COP (In: 21844, Out: 924, Total token: 22768)
   This message: $21.2585 COP (In: 926, Out: 37, Total token: 963) 
>> 🔎 Buscando documentos: definición de documento en derecho penal Colombia ...
>> 🧹 Docs filtrados. De 50 pasaron 22 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 8 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $610.7769999999999 COP (In: 26893, Out: 967, Total token: 27860)
   This message: $106.14900000000002 COP (In: 5049, Out: 43, Total token: 5092) 
>> ⏩️ Buscando los chunks siguientes de 20240813150814castrocuencamanualdederechopenalparteespecialtomoii2019pdf_chunk1141 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $752.1245000000001 COP (In: 33041, Out: 1216, Total token: 34257)
   This message: $141.34750000000003 COP (In: 6148, Out: 249, Total token: 6397) 
💬 Assistant:En el d

In [ ]:
messages = []
run_conversation("¿Cuándo se consuma el hurto?")

🙎‍♂️ User: ¿Cuándo se consuma el hurto?
>> 🤖 Generando respuesta ...
💰 Conversation price: $773.8545000000001 COP (In: 33963, Out: 1262, Total token: 35225)
   This message: $21.73 COP (In: 922, Out: 46, Total token: 968) 
>> 🔎 Buscando documentos: cuándo se consuma el hurto artículo 239 ley 599 del 2000 ...
>> 🧹 Docs filtrados. De 50 pasaron 23 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 4 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $862.4350000000001 COP (In: 37024, Out: 1682, Total token: 38706)
   This message: $88.5805 COP (In: 3061, Out: 420, Total token: 3481) 
💬 Assistant:El delito de hurto se consuma en el momento en que el sujeto activo logra extraer el bien de la esfera de dominio o custodia de la víctima, con la intención de obtener provecho para sí o para otro. No es necesario que el autor del delito logre materializar o realizar la utilidad o ganancia; basta con que el bien sea sacado de la esfera de dominio del dueño, po

In [ ]:
messages = []
run_conversation("¿Cuáles son las causales de agravación de la estafa?")

🙎‍♂️ User: ¿Cuáles son las causales de agravación de la estafa?
>> 🤖 Generando respuesta ...
💰 Conversation price: $2001.0460000000003 COP (In: 89155, Out: 2819, Total token: 91974)
   This message: $26.855 COP (In: 1004, Out: 102, Total token: 1106) 
>> 🔎 Buscando documentos: causales de agravación de la estafa artículo 267 ley 599 del 2000 ...
>> 🧹 Docs filtrados. De 50 pasaron 19 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 10 ...
>> 🔎 Buscando documentos: causales de agravación de la estafa ...
>> 🧹 Docs filtrados. De 50 pasaron 40 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 10 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $2306.4345 COP (In: 103932, Out: 2859, Total token: 106791)
   This message: $305.3885 COP (In: 14777, Out: 40, Total token: 14817) 
>> ⏩️ Buscando los chunks siguientes de 20240813150354universidadexternadoleccionesdederechopenalparteespecialvol12011pdf_chunk498 ...
>> 🤖 Gene

In [43]:
category_weight = (
    "category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"
)
search_semantic_query = (
    "causales de agravación de la estafa artículo 267 ley 599 del 2000"
)
search_text_query = f"content:causales de agravación artículo 267 AND title:ley 599 del 2000 AND {category_weight}"
# Add the category weight here
log(
    f"\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Text: {search_text_query}\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Semantic: {search_semantic_query}\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>"
)
content = search_legal_info_tool(
    text_query=search_text_query, semantic_query=search_semantic_query
)

>> 🔎 Buscando documentos: content:causales de agravación artículo 267 AND title:ley 599 del 2000 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🧹 Docs filtrados. De 50 pasaron 5 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 5 pasaron 3 ...


In [42]:
log_file_name = "logs/1 - Queries.log"


def ensure_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def log(message):
    global log_file_name
    ensure_directory("logs")
    with open(log_file_name, "a") as file:
        file.write(f"{datetime.now()}\n{message}\n")